In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)

True

In [2]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')    

challenge_question_prompt = """Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence.
Answer only with the question, no explanation."""

In [3]:
def challenge_question(challenge_question_prompt):
    messages = [
        {"role": "user", "content": challenge_question_prompt}
    ]

    challenge_question = OpenAI(api_key=openai_api_key).chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    ).choices[0].message.content


    display(Markdown(challenge_question))
    return challenge_question

In [4]:
models = ["gpt-4o-mini", "deepseek-chat", "gemini-2.0-flash", "llama-3.3-70b-versatile"]
api_urls = ["https://api.openai.com/v1/", "https://api.deepseek.com/v1", "https://generativelanguage.googleapis.com/v1beta/openai/", "https://api.groq.com/openai/v1"]
api_keys = [openai_api_key, deepseek_api_key, google_api_key, groq_api_key]

In [5]:
answers = []

def answer_challenge_question(model, url, api_key, challenge_question):
    messages = [{"role":"user", "content": challenge_question}]
    answer = OpenAI(api_key=api_key, base_url=url).chat.completions.create(
        model=model, 
        messages=messages
    ).choices[0].message.content
    answers.append(answer)


In [6]:
import threading

def ask_question_to_llm(challenge_question):
    for index in range(len(models)):
        thread = threading.Thread(target=answer_challenge_question, args=[models[index], api_urls[index], api_keys[index], challenge_question])
        thread.start()
        thread.join()

In [7]:

import dis


def judge_llms(challenge_question_prompt, answers):
    results = ''
    for index, answer in enumerate(answers):
        results += f"Response from competitor model: {models[index]}\n\n"
        results += answer + "\n\n"


    judge_prompt = f"""You are judging a competition between {len(models)} competitors.
    Each model has been given this question:

    {challenge_question_prompt}

    Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
    Respond with JSON, and only JSON, with the following format:
    {{"results": ["best competitor model", "second best competitor model", "third best competitor model", ...]}}

    Here are the responses from each competitor:

    {results}

    Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

    display(Markdown(judge_prompt))

    messages = [{"role": "user", "content": judge_prompt}]
    judge = OpenAI(api_key=openai_api_key).chat.completions.create(
        model="o3-mini", 
        messages=messages
    ).choices[0].message.content
    display(Markdown(judge))

In [ ]:
challenge_question = challenge_question(challenge_question_prompt)
ask_question_to_llm(challenge_question)
judge_llms(challenge_question_prompt=challenge_question_prompt, answers=answers)